In [1]:
import findspark


In [2]:
findspark.init()

In [4]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("samsung").setMaster("local")
sc = SparkContext(conf = conf)

In [16]:
rdf = sc.textFile(r"C:\Users\drang\Downloads\books_data\books.csv")

In [21]:
rdf.top(2)

['"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"',
 '"B0002K6K8O";"The Underground City";"Jules Verne";"0";"Digireads.com";"http://images.amazon.com/images/P/B0002K6K8O.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/B0002K6K8O.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/B0002K6K8O.01.LZZZZZZZ.jpg"']

In [19]:
rdf.count()

271380

In [24]:
#taking out only columns
new_1 = rdf.first()
columns1 = new_1.split(',')
columns1

['"ISBN";"Book-Title";"Book-Author";"Year-Of-Publication";"Publisher";"Image-URL-S";"Image-URL-M";"Image-URL-L"']

In [25]:
sc.stop()

In [28]:
#SQL DATAFRAME
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("samsung").master('local').getOrCreate()


In [29]:
type(spark)

pyspark.sql.session.SparkSession

In [31]:
bookdf = spark.read.csv(r"C:\Users\drang\Downloads\books_data\books.csv", header=True, inferSchema=True, sep =";")
bookdf.show(5)

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [32]:
bookdf.printSchema()

root
 |-- ISBN: string (nullable = true)
 |-- Book-Title: string (nullable = true)
 |-- Book-Author: string (nullable = true)
 |-- Year-Of-Publication: integer (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Image-URL-S: string (nullable = true)
 |-- Image-URL-M: string (nullable = true)
 |-- Image-URL-L: string (nullable = true)



In [33]:
userdf = spark.read.csv(r"C:\Users\drang\Downloads\books_data\users.csv", header=True, inferSchema=True, sep =";")
userdf.show(5)

+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|NULL|
|      2|stockton, califor...|  18|
|      3|moscow, yukon ter...|NULL|
|      4|porto, v.n.gaia, ...|  17|
|      5|farnborough, hant...|NULL|
+-------+--------------------+----+
only showing top 5 rows



In [34]:
userdf.printSchema()

root
 |-- User-ID: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: string (nullable = true)



In [40]:
bookdf.select('ISBN','Book-Title').show(5)

+----------+--------------------+
|      ISBN|          Book-Title|
+----------+--------------------+
|0195153448| Classical Mythology|
|0002005018|        Clara Callan|
|0060973129|Decision in Normandy|
|0374157065|Flu: The Story of...|
|0393045218|The Mummies of Ur...|
+----------+--------------------+
only showing top 5 rows



In [41]:
bookdf.select('Book-Title').distinct().count()

242154

In [108]:
ratingdf = spark.read.csv(r"C:\Users\drang\Downloads\books_data\ratings.csv", header=True, inferSchema=True, sep =";")
ratingdf.show(5)

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
+-------+----------+-----------+
only showing top 5 rows



In [109]:
ratingdf = ratingdf.withColumnRenamed("Book-Rating", "rating") # - is infered by sql as space

In [110]:
ratingdf = ratingdf.withColumnRenamed("User-ID", "userid")

In [69]:
userdf = userdf.withColumnRenamed("User-ID", "userid")

In [70]:
bookdf = bookdf.withColumnRenamed("Book-Title", "booktitle")

In [76]:
ratingdf.filter('rating >9').count()

78610

In [78]:
ratingdf.filter('rating >9').select('ISBN').show()

+----------+
|      ISBN|
+----------+
| 342310538|
|8440682697|
|3499230933|
|3596151465|
|055310666X|
|0395547032|
|0440414121|
|0060096195|
|0142302198|
|038076041X|
|0699854289|
|0786817070|
|0805057706|
|1573248533|
|3125785006|
|3423071516|
|3442413508|
|3442441080|
|3442446414|
|3506464078|
+----------+
only showing top 20 rows



In [82]:
userdf.count()

278859

In [83]:
userdf.dropDuplicates().count()

278859

In [135]:
userdf.select('Age').dropna("all").count() #or userdf.dropna()  same

278857

In [147]:
userdf.filter("Age == 'NULL'").count()

110761

In [139]:
ratingdf.agg({'rating' : 'max'}).show()

+-----------+
|max(rating)|
+-----------+
|         10|
+-----------+



In [95]:
ratingdf.groupby('rating').count().show()

+------+------+
|rating| count|
+------+------+
|     1|  1770|
|     6| 36924|
|     3|  5996|
|     5| 50974|
|     9| 67541|
|     4|  8904|
|     8|103736|
|     7| 76457|
|    10| 78610|
|     2|  2759|
|     0|716109|
+------+------+



In [89]:
ratingdf.groupby('rating').count().toPandas()

,rating,count
0,1,1770
1,6,36924
2,3,5996
3,5,50974
4,9,67541
5,4,8904
6,8,103736
7,7,76457
8,10,78610
9,2,2759


In [97]:
ratingdf.orderBy('rating').show(5)

+------+----------+------+
|userid|      ISBN|rating|
+------+----------+------+
|276746|0425115801|     0|
|276746|0786014512|     0|
|276746|0449006522|     0|
|276727|0446520802|     0|
|276746|0553561618|     0|
+------+----------+------+
only showing top 5 rows



In [98]:
ratingdf.orderBy(ratingdf.rating.desc()).show()

+------+----------+------+
|userid|      ISBN|rating|
+------+----------+------+
|276822|0699854289|    10|
|276847|3551551677|    10|
|276822|0786817070|    10|
|276760|8440682697|    10|
|276822|0805057706|    10|
|276772|3596151465|    10|
|276822|1573248533|    10|
|276808|0395547032|    10|
|276835|3125785006|    10|
|276822|0060096195|    10|
|276847|3423071516|    10|
|276847|3551551685|    10|
|276847|3442413508|    10|
|276745| 342310538|    10|
|276847|3442441080|    10|
|276788|055310666X|    10|
|276847|3442446414|    10|
|276822|0142302198|    10|
|276847|3506464078|    10|
|276772|3499230933|    10|
+------+----------+------+
only showing top 20 rows



In [118]:
new = ratingdf.withColumn('rate100', ratingdf.rating*10)

In [120]:
ratingdf.show(5)

+------+----------+------+
|userid|      ISBN|rating|
+------+----------+------+
|276725|034545104X|     0|
|276726|0155061224|     5|
|276727|0446520802|     0|
|276729|052165615X|     3|
|276729|0521795028|     6|
+------+----------+------+
only showing top 5 rows



In [119]:
new.show(5)

+------+----------+------+-------+
|userid|      ISBN|rating|rate100|
+------+----------+------+-------+
|276725|034545104X|     0|      0|
|276726|0155061224|     5|     50|
|276727|0446520802|     0|      0|
|276729|052165615X|     3|     30|
|276729|0521795028|     6|     60|
+------+----------+------+-------+
only showing top 5 rows



In [121]:
bookdf

DataFrame[ISBN: string, booktitle: string, Book-Author: string, Year-Of-Publication: int, Publisher: string, Image-URL-S: string, Image-URL-M: string, Image-URL-L: string]

In [124]:
ratingdf.join(bookdf, ratingdf.ISBN == bookdf.ISBN).select('booktitle','rating').orderBy(ratingdf.rating.desc()).show(5)

+--------------------+------+
|           booktitle|rating|
+--------------------+------+
|Little Wolf's Boo...|    10|
|        The Romantic|    10|
|Glue (First Facts...|    10|
| The Moon of Gomrath|    10|
|The Brambly Hedge...|    10|
+--------------------+------+
only showing top 5 rows

